In [1]:
import pandas as pd
import numpy as np
import random
import cv2
import numpy as np
import matplotlib.pyplot as plt
import json
from IPython.display import Image
import shutil
import copy
from numpy import int32
import os
from numpy import float32
import cv2
import pytesseract
pytesseract.pytesseract.tesseract_cmd=r"C:/Program Files/Tesseract-OCR/tesseract.exe"
import re

In [6]:
#YOLO_classes
#0=chart_title
#1=x_axis_label
#2=y_axis_label
#3=legend_elements

int_list_images=[]
#Path of Tested Images
list_images=os.listdir('C:/Users/romil/Desktop/MCM Practicum/detect/exp5/exp5/')
#print(list_images)
for i in list_images:
    int_list_images.append(int(i.replace('.png','')))

recreated_annotation_list=[]

for i in int_list_images:
    legend_items=[]
    image_dict={'image_index':i}
    #Path of TXT files for Tested Imagea
    with open('C:/Users/romil/Desktop/MCM Practicum/detect/exp5/labels/'+str(i)+'.txt','r') as dump:
        #Path of tested images
        img_path='C:/Users/romil/Desktop/MCM Practicum/detect/exp5/exp5/'+str(i)+'.png'
        image=cv2.imread(img_path, 0)
        thresh=255-cv2.threshold(image, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        lines=dump.readlines()
        x_axis_label_flag=0
        chart_title_flag=0
        y_axis_label_flag=0
        for line in enumerate(lines):
            height,width=cv2.imread(img_path).shape[:2]
            line=line[1].split(' ')
            class_id=int(float(line[0]))
            class_id, x_yolo,y_yolo,width_yolo,height_yolo=line[0:]
            x_yolo,y_yolo,width_yolo,height_yolo,class_id=float(x_yolo),float(y_yolo),float(width_yolo),float(height_yolo),int(float(class_id)) 
            
            #Extracting text from x_axis_label
            if class_id==1 and x_axis_label_flag==0:
                x_axis_label_flag=1
                h=round(height_yolo*height)
                w=round(width_yolo*width)
                x=round(x_yolo*width-(w/2))
                y=round(y_yolo*height-(h/2))
                cropped_bbox=thresh[y:y+h,x:x+w]
                x_axis=pytesseract.image_to_string(cropped_bbox, lang='eng',config='--psm 6')
                x_axis=x_axis.replace('%','Percentage')
                x_axis=re.sub(r"[^a-zA-Z0-9]+", ' ', x_axis)
                x_axis=x_axis.strip()
                x_axis_dict={'x_axis_label':x_axis}

            #Extracting text from y_axis_label
            if class_id==2 and y_axis_label_flag==0:
                y_axis_label_flag=1
                h=round(height_yolo*height)
                w=round(width_yolo*width)
                x=round(x_yolo*width-(w/2))
                y=round(y_yolo*height-(h/2))
                cropped_bbox=thresh[y:y+h,x:x+w]
                #Rotating images to recognize vertical text on y axis
                cropped_bbox=cv2.rotate(cropped_bbox, cv2.ROTATE_90_CLOCKWISE)
                y_axis=pytesseract.image_to_string(cropped_bbox, lang='eng',config='--psm 6')
                y_axis=y_axis.replace('%','Percentage')
                y_axis=re.sub(r"[^a-zA-Z0-9]+", ' ', y_axis)
                y_axis=y_axis.strip()
                y_axis_dict={'y_axis_label':y_axis}
            
            #Extracting text from chart_title
            if class_id==0 and chart_title_flag==0:
                chart_title_flag=1
                h=round(height_yolo*height)
                w=round(width_yolo*width)
                x=round(x_yolo*width-(w/2))
                y=round(y_yolo*height-(h/2))
                cropped_bbox=thresh[y:y+h,x:x+w]
                chart_title=pytesseract.image_to_string(cropped_bbox, lang='eng',config='--psm 6')
                chart_title=re.sub(r"[^a-zA-Z0-9]+", ' ', chart_title)
                chart_title=chart_title.strip()
                chart_dict={'chart_title':chart_title}
            elif chart_title_flag==0:
                chart_dict={'chart_title':'not recognized by model'}
            
            #Extracting text from legend items
            if class_id==3:
                h=round(height_yolo*height)
                w=round(width_yolo*width)
                x=round(x_yolo*width-(w/2))
                y=round(y_yolo*height-(h/2))
                cropped_bbox=thresh[y:y+h,x:x+w]
                legend_item=pytesseract.image_to_string(cropped_bbox, lang='eng',config='--psm 6')
                legend_item=re.sub(r"[^a-zA-Z0-9]+", ' ', legend_item)
                legend_item=legend_item.strip()
                if legend_item not in legend_items:
                    legend_items.append(legend_item)

    legend_dict={'legend_items':legend_items}
    recreated_annotation_list.append(image_dict | chart_dict | x_axis_dict | y_axis_dict | legend_dict)

#print((recreated_annotation_list))

In [7]:
type(recreated_annotation_list)
import json
with open('extracted_text(IoU 0.8).json', 'w') as f:
    json.dump(recreated_annotation_list, f)